In [4]:
%matplotlib widget
import numpy as np
import pandas as pd
import random
import talib as ta
import math
from importlib import reload 
# from numba import jit

from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
vTest = 1000

''' 清空 stock_cal table '''
sql = "truncate table day_zhibiao"
basic.exec_sql(engine, sql)

data_dir = '../data/'
# ts_list = pd.read_csv( '../data/sl_hs_const.csv')
ts_list = pd.read_csv( data_dir + 'ts_all.csv')
# ts_list = pd.read_csv( data_dir + 'stocklist.csv')

# 从 table 中读出来
# sql="SELECT * FROM sl_shengutong"
# ts_list = pd.read_sql_query(sql, conn)
ts_list_size = len( ts_list )
ts_list.head(3), ts_list.shape 


(   Unnamed: 0    ts_code  symbol  name area industry market  list_date
 0           0  000001.SZ       1  平安银行   深圳       银行     主板   19910403
 1           1  000002.SZ       2   万科A   深圳     全国地产     主板   19910129
 2           2  000004.SZ       4  国华网安   深圳     软件服务     主板   19910114,
 (4760, 8))

In [5]:
# ts_list.head(2), len(ts_list)

In [6]:
''' 运算 percentage st  '''

# K线分析：引线+实体部分
def cal_upper(df):
    return ( df['high'] - max( df['open'], df['close'] ) ) / df['pre_close'] * 100
def cal_down(df):  
    return ( min( df['open'], df['close'] ) - df['low']  ) / df['pre_close'] * 100
def cal_range(df):  # 振幅%
    return ( df['high'] - df['low']  ) / df['pre_close'] * 100
def cal_st_high(df):  
    return df['open'] if df['open'] >= df['close'] else df['close']
def cal_st_low(df):
    return df['open'] if df['open'] < df['close'] else df['close']
def cal_st(df):  # 实体振幅
    return abs( df['open'] - df['close'] )
def cal_st_range(df):  # 实体振幅%
    return abs( df['open'] - df['close']  ) / df['pre_close'] * 100
# apply 进函数
def cal_pct_st():
    df['pct_upper'] = df.apply(cal_upper, axis=1)    # 上引线百分比
    df['pct_down']  = df.apply(cal_down,  axis=1)    # 下引线百分比
    df['pct_range'] = df.apply(cal_range, axis=1)    # 振幅百分比
    df['pct_st_range']  = df.apply(cal_st_range, axis=1)   # 实体部分百分比
    df['st_high']   = df.apply(cal_st_high,  axis=1)   # 实体高点价格
    df['st_low']    = df.apply(cal_st_low,   axis=1)   # 实体低点价格
    # df['pct_st']  = df.apply(cal_st, axis=1)   # 实体高度
    # 星线定义
    df['xing'] = df['pct_st_range']<=1
    return 'pct . '

def cal_ma():
    # yong ta-lib 包实现
    df.insert( df.shape[1], 'ma3', ta.MA(df['close'], 3) )
    df.insert( df.shape[1], 'ma5', ta.MA(df['close'], 5) )
    df.insert( df.shape[1], 'ma10', ta.MA(df['close'], 10) )
    df.insert( df.shape[1], 'ma20', ta.MA(df['close'], 20) )
    df.insert( df.shape[1], 'ma30', ta.MA(df['close'], 30) )
    df.insert( df.shape[1], 'ma60', ta.MA(df['close'], 60) )
    df.insert( df.shape[1], 'ma120', ta.MA(df['close'], 120) )
    df.insert( df.shape[1], 'ma250', ta.MA(df['close'], 250) )
    return 'ma . '

def cal_macd():
    ''' way2 : ta-lib '''
    close = df.close.values  # 为 MACD 数据初始化close
    df['diff'], df['dea'], df['bar'] = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    return 'macd . '


In [7]:
''' 计算macd '''
# 输入df_name,算法参数。其中ewm函数算法快300-500倍
# def cal_macd2(dfIn, macd_way=2, ema_fast=12, ema_slow=26, dea_day=9):
#     dfIn['ema_fast'], dfIn['ema_slow'] = 0.0, 0.0
#     dfIn['diff'], dfIn['dea'], dfIn['bar'] = 0.0,0.0,0.0
#     dfIn['ema_fast'] = dfIn['close'].ewm(span=ema_fast,     adjust=False).mean()
#     dfIn['ema_slow'] = dfIn['close'].ewm(span=ema_slow,     adjust=False).mean()
#     dfIn['diff'] = dfIn['ema_fast'] - dfIn['ema_slow']
#     dfIn['dea'] = dfIn['diff'].ewm(span=9, adjust=False).mean()
#     dfIn['bar'] = (dfIn['diff'] - dfIn['dea']) * 2
#     return dfIn
    
# def cal_more():
    # # 股价变化均值
    # df.insert( df.shape[1], 'ma3_pct_change', ta.MA(df['pct_change'], 3) )
    # # N天之内最大股价
    # df['MAX3_pct_change']=df['pct_change'].rolling(3).max()
    # # n天之内最高收盘价
    # df['MAX90_high']=df['high'].rolling(90).max()

# def cal_boll():
#     df['bb-u'], df['bb-m'], df['bb-l'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
#     df['upper'], df['middle'], df['lower'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )



' 计算macd '

In [8]:
""" 均线系列函数， """

# 计算在60线以下的days
def jx_ud60_count(val):
    if val <= 0:
        jx_ud60_count.count += 1
    else:
        jx_ud60_count.count = 0
    return jx_ud60_count.count

# 均线的斜率, 输出角度（不是度数）
def cal_jx_xl():
    df['jx_xl_5']  = (df['ma5']  - df['ma5'].rolling(2).mean()) * 2/  df['ma5']
    df['jx_xl_10'] = (df['ma10'] - df['ma10'].rolling(2).mean()) * 2/ df['ma10']
    df['jx_xl_20'] = (df['ma20'] - df['ma20'].rolling(2).mean()) * 2/ df['ma20']
    df['jx_xl_30'] = (df['ma30'] - df['ma30'].rolling(2).mean()) * 2/ df['ma30']
    df['jx_xl_60'] = (df['ma60'] - df['ma60'].rolling(2).mean()) * 2/ df['ma60']
    df['jx_xl_120'] = (df['ma120'] - df['ma120'].rolling(2).mean()) * 2/ df['ma120']
    df['jx_xl_250'] = (df['ma250'] - df['ma250'].rolling(2).mean()) * 2/ df['ma250']
    return  'xl . '

# 计算站上5日均线的天数
def jx_zs(val):
    if val >= 0:
        jx_zs.count += 1
    else:
        jx_zs.count = 0
    return jx_zs.count   

def cal_jx_zs_all():
    
    # jx - 站上5日均线天数
    df['zs_status'] = df['close'] - df['ma5']
    jx_zs.count = 0
    df['jx_zs_5'] = df['zs_status'].apply(jx_zs) #new column in dataframe
    # jx - 站上10日均线天数
    df['zs_status'] = df['close'] - df['ma10']
    jx_zs.count = 0
    df['jx_zs_10'] = df['zs_status'].apply(jx_zs)
    # jx - 站上20日均线天数
    df['zs_status'] = df['close'] - df['ma20']
    jx_zs.count = 0
    df['jx_zs_20'] = df['zs_status'].apply(jx_zs)
    # jx - 站上30日均线天数
    df['zs_status'] = df['close'] - df['ma30']
    jx_zs.count = 0
    df['jx_zs_30'] = df['zs_status'].apply(jx_zs)
    # print( end='30.' )
    
    # jx - 站上60日均线天数
    df['zs_status'] = df['close'] - df['ma60']
    jx_zs.count = 0
    df['jx_zs_60'] = df['zs_status'].apply(jx_zs)
    # print( end='60.' )
    # jx - 站上120日均线天数
    df['zs_status'] = df['close'] - df['ma120']
    jx_zs.count = 0
    df['jx_zs_120'] = df['zs_status'].apply(jx_zs)
    # print( end='120.' )
    # jx - 站上250日均线天数
    df['zs_status'] = df['close'] - df['ma250']
    jx_zs.count = 0
    df['jx_zs_250'] = df['zs_status'].apply(jx_zs)
    del df['zs_status']
    
    return('jx_zs 5-250. ')

In [9]:
""" 量价关系函数 """
# 成交量、x日均量
def cal_vol_ma():
    df['vol_ma3']=df['volume'].rolling(3).mean()
    df['vol_ma5']=df['volume'].rolling(5).mean()
    df['vol_ma10']=df['volume'].rolling(10).mean()
    df['vol_ma20']=df['volume'].rolling(20).mean()
    df['vol_ma30']=df['volume'].rolling(30).mean()
    df['vol_ma60']=df['volume'].rolling(60).mean()
    return 'vol_ma . '

# 成交量、x日均量
def cal_lj_fl():
    df['lj_fl_1_3']=df['volume']/df['vol_ma3']
    df['lj_fl_3_10']=df['vol_ma3']/df['vol_ma10']
    df['lj_fl_5_20']=df['vol_ma5']/df['vol_ma20']
    df['lj_fl_5_60']=df['vol_ma5']/df['vol_ma60']
    df['lj_fl_20_60']=df['vol_ma20']/df['vol_ma60']
    # df['vol_ma60']=df['volume'].rolling(60).mean()
    return 'lj_fl . '



In [10]:
%%time

print('计算各类指标 : ')
for i in range( len(ts_list)  ):
    
    # begin with ts_code
    print(ts_list.index[i]+1,' - ',str(ts_list_size), ts_list['ts_code'][i], end=" ")
    
    # 不计算8字头，北京股票
    if ts_list['ts_code'][i].startswith('8'):
        print( 'ignore 8xxx', end='\n' )
        continue
    # 不计算688字头，科创板
    if ts_list['ts_code'][i].startswith('688'):
        print( 'ignore 688x', end='\n' )
        continue
    
    # 本地MySQL读取
    sql = "select * from tran_daily where ts_code='"+ts_list['ts_code'][i]+"'"
    df = basic.read_data(conn, sql)
    df.set_index('trade_date', drop=True, append=False, inplace=True)
    df = df.sort_index(ascending=True)
    # 如果没有交易记录，则跳过loop
    if len(df) == 0:
        print( 'no data', end='' )
        continue  
    
    # 计算 percentage 和 实体部分
    print( cal_pct_st(), end='' )
    # 计算ma
    print( cal_ma(), end='' )
    # 计算macd
    print( cal_macd(), end='' )
    

    # 计算junxian - 60线以下天数
    df['ud60_status'] = df['close'] - df['ma60']
    jx_ud60_count.count = 0
    df['jx_days_ud60'] = df['ud60_status'].apply(jx_ud60_count) #new column in dataframe
    del df['ud60_status']
    print( end='jx_ud60 . ' )

    # 计算cal_jx_xl - ma5,10,...250 均线斜率 
    print( cal_jx_xl(), end='' )
    # jx - 站上 x 日均线的天数
    print( cal_jx_zs_all(), end='' )

    """ 计算量价关系 """
    print( cal_vol_ma(), end='' )
    # def cal_vol_ma():
    # 计算放量情况
    print( cal_lj_fl(), end='' )
  
    """ 处理验证列， """
    # 后n天上涨率
    # if ts_list['ts_code'][i] == '300474.SZ':  # for test purpose
    #     df['pct_change'] = [ random.randint(-2,2) for i in range(len(df)) ]
    df = df.sort_index(ascending=False)  # 倒着计算 sum 
    df['sz_sum_5'] = df['pct_change'].rolling(5+1).sum()  - df['pct_change']
    df['sz_sum_10'] = df['pct_change'].rolling(10+1).sum()  - df['pct_change']
    df['sz_sum_20'] = df['pct_change'].rolling(20+1).sum()  - df['pct_change']
    print( 'sz_sum. ', end='' )
    # 后n天最大上涨率    
    df['close_max'] = df['close'].rolling(5+1).max()
    df['sz_max_5'] = (df['close_max']-df['close'])*100/df['close']
    df['close_max'] = df['close'].rolling(10+1).max()
    df['sz_max_10'] = (df['close_max']-df['close'])*100/df['close']
    df['close_max'] = df['close'].rolling(20+1).max()
    df['sz_max_20'] = (df['close_max']-df['close'])*100/df['close']
    del df['close_max']
    print( 'sz_max. ', end='' )
    
    """ Final : 空值填充为0 """ 
    df = df.fillna(value=0)

    """ 输出 to csv """
    # df.to_csv( data_dir+'cal/' + ts_list['ts_code'][i] +'.cal.csv', index=True )    
    ''' 输出到 mysql '''
    sql = "delete from day_zhibiao where ts_code='"+ts_list['ts_code'][i] +"'"
    basic.exec_sql(conn, sql)
    # df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000)
    write_no = df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000) # 20000个，每天成交几千只股票，足够了
    # index=True, 需要输出日期字段    
    # 首次运行，建立stock_cal表, 可以从df中建立表，表之前不存在，然后再改字段和index

    print( end='\n')

print('\n\n指标运算结束')

计算各类指标 : 
1  -  4760 000001.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
2  -  4760 000002.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
3  -  4760 000004.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
4  -  4760 000005.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
5  -  4760 000006.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
6  -  4760 000007.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
7  -  4760 000008.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
8  -  4760 000009.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
9  -  4760 000010.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_fl . sz_sum. sz_max. 
10  -  4760 000011.SZ pct . ma . macd . jx_ud60 . xl . jx_zs 5-250. vol_ma . lj_f

In [ ]:
# df.info()
df.head(3)


,ts_code,open,high,low,close,pre_close,change,pct_change,volume,value,...,lj_fl_3_10,lj_fl_5_20,lj_fl_5_60,lj_fl_20_60,sz_sum_5,sz_sum_10,sz_sum_20,sz_max_5,sz_max_10,sz_max_20
trade_date,,,,,,,,,,,,,,,,,,,,,
20230719,300761.SZ,19.08,19.14,18.80,19.03,19.12,-0.09,-0.4707,11376.5,21559.0,...,0.945523,0.825260,0.720630,0.873215,0.0,0.0,0.0,0.0,0.0,0.0
20230718,300761.SZ,18.80,19.18,18.69,19.12,18.89,0.23,1.2176,18373.1,34991.6,...,0.896008,0.811744,0.720242,0.887277,0.0,0.0,0.0,0.0,0.0,0.0
20230717,300761.SZ,18.53,18.95,18.33,18.89,18.62,0.27,1.4501,12895.0,24272.1,...,0.752417,0.757740,0.675306,0.891210,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 调试：之后 n 天内的最大涨幅 -- reserved 

df = df.sort_index(ascending=False)
df['close_max'] = df['close'].rolling(5+1).max()
df['sz_max_5'] = (df['close_max']-df['close'])/df['close']
df.head(100)[['close', 'close_max', 'sz_max_5']]

,close,close_max,sz_max_5
trade_date,,,
20230719,19.03,NaN,NaN
20230718,19.12,NaN,NaN
20230717,18.89,NaN,NaN
20230714,18.62,NaN,NaN
20230713,18.76,NaN,NaN
...,...,...,...
20230228,38.72,40.86,0.055269
20230227,37.89,40.17,0.060174
20230224,38.05,39.62,0.041261


In [ ]:
# 调试：累计涨幅 -- reserved 
# if ts_list['ts_code'] == '300474.SZ':
# df['pct_change'] = [ random.randint(-2,2) for i in range(len(df)) ]
# df = df.sort_index(ascending=False)
# df['pct_sum_5'] = df['pct_change'].rolling(5+1).sum()  - df['pct_change']
# df.head(15)[['pct_change', 'pct_sum_5']]

In [ ]:
# received ...
# df['test'] = m_list = [random.randint(0,3) for i in range(1693)]
# df['test'] 
# df['pct_sum'] = df['test'].rolling(4).sum()  - df['test']
# df.head(12)[['test', 'pct_sum']]
# df = df.sort_index(ascending=False)



In [ ]:
# df1 = df[ df['jx_days_ud60']==0 ]
# df1.head()
#筛选a列中值为a的行，形成新的df1
# df1 = df[ df['a'].isin(['a']) ]
#筛选列名为a、b、c的列，形成新的df1
# df1 = df[['a','b','c']]


In [ ]:
# 插入和删除行，很有用
# df.insert( df.shape[1], 'ma250', ta.MA(df['close'], 250) )
# df.drop( labels=['MA125'], axis=1, inplace=True)
# df.head(3)